In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms

from torch.autograd import Variable

In [2]:
transform = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

trainset = torchvision.datasets.CIFAR10(root = "./cifar10-data",
                                       train = True, transform = transform , download = True)
trainloader = torch.utils.data.DataLoader(trainset , batch_size = 32,
                                         shuffle = True , num_workers = 3 , pin_memory=True)

testset = torchvision.datasets.CIFAR10(
    root = "./cifar10-data" , train = False , download = True, transform = transform
)

testloader = torch.utils.data.DataLoader(
    testset , batch_size = 4 , shuffle = False , num_workers=2 , pin_memory=True
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
cuda = True
class AlexNet(nn.Module):
    def __init__(self , num_classes = 1000):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(
                3 , 32 , kernel_size = 4 , padding = 1 , stride = 1
            ),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2 , stride = 2),
            nn.Conv2d(
                32 , 64 , kernel_size = 3 , padding = 1, stride = 1
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 2 , stride = 2),
            nn.Conv2d(
                64,128, kernel_size = 3 , padding = 1 , stride = 1
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(
                128,256,kernel_size = 3, padding = 1, stride = 1
            ),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(
                256, 128, kernel_size = 3 , stride = 1 , padding = 1
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 2 ),
        )
        self.classification = nn.Sequential(
            nn.Dropout(),
            nn.Linear(128*3*3, 512),
            nn.ReLU(inplace= True),
            nn.Dropout(),
            nn.Linear(512,10),
            nn.ReLU(inplace = True)
        )
    def forward(self , x):
        x = self.features(x)
        x = x.view(x.size(0) , 128*3*3)
        x = self.classification(x)
        return x
model = AlexNet()

if cuda:
    model.cuda()
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (10): ReLU()
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (13): ReLU()
    (14): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (16)

In [4]:
data = iter(trainloader)

image , label = data.next()

if cuda:
    image , label = image.cuda(), label.cuda()
image , label = Variable(image) , Variable(label)

In [16]:
image.shape

torch.Size([32, 3, 32, 32])

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters() , lr = 0.001 , weight_decay=1e-5 , momentum=0.9 )

In [7]:
for epoch in range(30):
    running_loss = 0
    
    for i, data in enumerate(trainloader , 0):
        inputs,labels = data
        
        if cuda:
            inputs , labels = inputs.cuda() , labels.cuda()
        inputs , labels = Variable(inputs) , Variable(labels)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        
        if i%200 == 0:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print("Finished Training")

[1,     1] loss: 0.001
[1,   201] loss: 0.222
[1,   401] loss: 0.189
[1,   601] loss: 0.169
[1,   801] loss: 0.163
[1,  1001] loss: 0.155
[1,  1201] loss: 0.147
[1,  1401] loss: 0.143
[2,     1] loss: 0.001
[2,   201] loss: 0.137
[2,   401] loss: 0.134
[2,   601] loss: 0.131
[2,   801] loss: 0.129
[2,  1001] loss: 0.125
[2,  1201] loss: 0.123
[2,  1401] loss: 0.121
[3,     1] loss: 0.001
[3,   201] loss: 0.116
[3,   401] loss: 0.115
[3,   601] loss: 0.115
[3,   801] loss: 0.114
[3,  1001] loss: 0.109
[3,  1201] loss: 0.111
[3,  1401] loss: 0.109
[4,     1] loss: 0.000
[4,   201] loss: 0.105
[4,   401] loss: 0.102
[4,   601] loss: 0.101
[4,   801] loss: 0.102
[4,  1001] loss: 0.102
[4,  1201] loss: 0.101
[4,  1401] loss: 0.098
[5,     1] loss: 0.001
[5,   201] loss: 0.096
[5,   401] loss: 0.097
[5,   601] loss: 0.094
[5,   801] loss: 0.094
[5,  1001] loss: 0.094
[5,  1201] loss: 0.092
[5,  1401] loss: 0.091
[6,     1] loss: 0.000
[6,   201] loss: 0.089
[6,   401] loss: 0.088
[6,   601] 

In [8]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    
    if cuda:
        images , labels = images.cuda() , labels.cuda()
    model.eval()
    outputs = model(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
100 * correct / total))

Accuracy of the network on the 10000 test images: 82 %


In [9]:
classes = ('plane', 'car', 'bird', 'cat',
'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    
    if cuda:
        images , labels = images.cuda() , labels.cuda()
    model.eval()
    outputs = model(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 85 %
Accuracy of   car : 90 %
Accuracy of  bird : 70 %
Accuracy of   cat : 60 %
Accuracy of  deer : 78 %
Accuracy of   dog : 79 %
Accuracy of  frog : 88 %
Accuracy of horse : 90 %
Accuracy of  ship : 91 %
Accuracy of truck : 88 %
